In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR100
import torchvision.transforms as TF
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset
from torchvision.utils import make_grid
from tqdm.notebook import tqdm as tqdm
from random import randint

%matplotlib inline

In [ ]:
transform = TF.Compose([TF.ToTensor(), TF.Normalize((0.5074,0.4867,0.4411),(0.2011,0.1987,0.2025))])
device = "cuda" if torch.cuda.is_available() else "cpu"

TRAIN_DATA = CIFAR100("./data", train=True, download=True, transform=transform)
TEST_DATA = CIFAR100("./data", train=False, download=True, transform=transform)

In [ ]:
img, label  = TRAIN_DATA[0]
print("Image Shape: ", img.shape)
print("Image: ", img)
print("Label: ", label)

In [ ]:
images = []
labels = []
for i in range(25):
  image, label = TRAIN_DATA[i]
  images.append(image)
  labels.append(label)

grid_image = make_grid(images, nrow=5, padding=5).permute(1, 2, 0)
grid_image = np.clip(grid_image.numpy(), 0 , 1)
plt.figure(figsize=(7.5, 15))
plt.axis("off")
plt.imshow(grid_image)

In [ ]:
def initialize_parameters(model):

  def initialize_layer(layer):
    if isinstance(layer, nn.Conv2d):
      nn.init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='relu')

    if isinstance(layer, nn.Linear):
      nn.init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='relu')

  model.apply(initialize_layer)


def accuracy(output, labels):
  _, predictions = torch.max(output, dim=1)
  return torch.sum(predictions == labels).item() / len(predictions)

In [ ]:
def plot_show(epoch_loss, val_loss, val_acc):
  plt.figure(figsize=(10, 12))

  plt.subplot(2, 1, 1)
  plt.plot(epoch_loss, label="Training")
  plt.plot(val_loss, label="Validation")
  plt.title("Loss History")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()

  plt.subplot(2, 1, 2)
  plt.plot(val_acc)
  plt.title("Accuracy History")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy %")

  plt.show()

In [ ]:
def fit(model, optimizer, scheduler, train_loader, val_loader, epochs):
  EPOCH_LOSS = []
  VAL_LOSS = []
  VAL_ACC = []

  for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    nsteps = 0

    for batch in tqdm(train_loader):
      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)
      nsteps += 1

      optimizer.zero_grad()
      output = model(images)
      loss = F.cross_entropy(output, labels)
      loss.backward()
      epoch_loss += loss.item()
      optimizer.step()
      scheduler.step()

    EPOCH_LOSS.append(epoch_loss / nsteps)

    with torch.no_grad():
      model.eval()

      val_loss = 0
      val_acc = 0
      nsteps = 0

      for batch in val_loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        nsteps += 1

        prediction = model(images)
        loss = F.cross_entropy(prediction, labels)
        acc = accuracy(prediction, labels)
        val_loss += loss.item()
        val_acc += acc
      
      VAL_LOSS.append(val_loss / nsteps)
      VAL_ACC.append(val_acc / nsteps * 100)

      model.train()
    
  return EPOCH_LOSS, VAL_LOSS, VAL_ACC

In [ ]:
base_model = torchvision.models.resnet34(pretrained=False, progress=False)
base_model.fc = nn.Sequential(nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 100))
initialize_parameters(base_model)

base_model.to(device)
print(base_model)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 100
LEARNING_RATE = 0.001
REGULARIZATION = 0.001

train_loader = DataLoader(TRAIN_DATA, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TEST_DATA, batch_size=BATCH_SIZE)
optimizer = torch.optim.Adam(base_model.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1)

simple_model_data = fit(base_model, optimizer, scheduler, train_loader, val_loader, EPOCHS)

plot_show(*simple_model_data)

In [ ]:
print("Accuracy of Base Model: ", simple_model_data[2][-1])

In [ ]:
class RotationalTransform:
    def __init__(self, angle):
        self.angle = angle

    def __call__(self, x):
        return TF.transforms.F.rotate(x, self.angle)

class VerticalFlip:
    def __init__(self):
        pass
    def __call__(self, x):
        return TF.transforms.F.vflip(x)

class HorizontalFlip:
    def __init__(self):
        pass
    def __call__(self, x):
        return TF.transforms.F.hflip(x)

In [ ]:
class SelfSupervisedDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.class_transforms = [RotationalTransform(0), 
                                 RotationalTransform(90), 
                                 RotationalTransform(180), 
                                 RotationalTransform(270), 
                                 HorizontalFlip(), 
                                 VerticalFlip()]
        self.images = []
        self.labels = []
        for img, _ in self.data:
          label = randint(0, len(self.class_transforms) - 1)
          self.images.append(self.class_transforms[label](img))
          self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return len(self.images)

SELF_SUPERVISED_TRAIN_DATA = SelfSupervisedDataset(TRAIN_DATA)
SELF_SUPERVISED_TEST_DATA = SelfSupervisedDataset(TEST_DATA)

In [ ]:
images = []
labels = []
for i in range(25):
  image, label = SELF_SUPERVISED_TRAIN_DATA[i]
  images.append(image)
  labels.append(label)

grid_image = make_grid(images, nrow=5, padding=5).permute(1, 2, 0)
grid_image = np.clip(grid_image.numpy(), 0 , 1)
plt.figure(figsize=(7.5, 15))
plt.axis("off")
plt.imshow(grid_image)

In [ ]:
supervised_model = torchvision.models.resnet34(pretrained=False, progress=False)
supervised_model.fc = nn.Sequential(nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 6))

supervised_model.to(device)
print(supervised_model)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 25
LEARNING_RATE = 0.001
REGULARIZATION = 0.001

train_loader = DataLoader(SELF_SUPERVISED_TRAIN_DATA, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(SELF_SUPERVISED_TEST_DATA, batch_size=BATCH_SIZE)
optimizer = torch.optim.Adam(supervised_model.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1)

supervised_model_data_pre = fit(supervised_model, optimizer, scheduler, train_loader, val_loader, EPOCHS)

plot_show(*supervised_model_data_pre)

In [ ]:
supervised_model.fc[2] = nn.Linear(256, 100).to(device)

print(supervised_model)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 25
LEARNING_RATE = 0.001
REGULARIZATION = 0.001

train_loader = DataLoader(TRAIN_DATA, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TEST_DATA, batch_size=BATCH_SIZE)
optimizer = torch.optim.Adam(supervised_model.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1)

supervised_model_data = fit(supervised_model, optimizer, scheduler, train_loader, val_loader, EPOCHS)

plot_show(*supervised_model_data)

In [ ]:
print("Accuracy of Supervised Model: ", supervised_model_data[2][-1])